# Doing an exploration on recent disaster tweets for a disaster category

In [1]:
import pandas as pd
import pickle

import spacy
from spacy import displacy
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
import networkx as nx 
import nx_altair as nxa
from collections import Counter
import warnings

from tools import extract_assign_location, disaster_title, topNlocations, top_locations, \
                    sampleNtweets, top_locations_tweets, show_top_locations, get_top_locations_flatten, \
                    get_location_graph_data, build_network, get_layout, plot_disaster_mentions, is_intersect, \
                    show_tweets_locations, top_location_pairs

warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', 280)

## Define any upstream dependencies

In [2]:
upstream = [
    'vectorizer',
    'category_classification_models',
    'recent_tweets_wildfire',
    'recent_tweets_earthquake',
    'recent_tweets_flood',
    'recent_tweets_hurricane'
]
random_seed = None
disaster_kind = None

In [3]:
# Parameters
random_seed = 42
disaster_kind = "recent_tweets_hurricane"
upstream = {
    "vectorizer": {
        "nb": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/vectorizer.ipynb",
        "vectorizer": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/vectorizer.pkl",
        "vocab": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/vocab.pkl",
    },
    "category_classification_models": {
        "nb": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/category_classification_models.ipynb",
        "model_lr": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/model_lr.pkl",
        "model_rf": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/model_rf.pkl",
        "model_nb": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/model_nb.csv",
        "model_votingc": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/model_votingc.csv",
    },
    "recent_tweets_earthquake": {
        "nb": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/recent_tweets_earthquake.ipynb",
        "file": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/recent_tweets_earthquake.csv",
    },
    "recent_tweets_wildfire": {
        "nb": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/recent_tweets_wildfire.ipynb",
        "file": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/recent_tweets_wildfire.csv",
    },
    "recent_tweets_hurricane": {
        "nb": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/recent_tweets_hurricane.ipynb",
        "file": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/recent_tweets_hurricane.csv",
    },
    "recent_tweets_flood": {
        "nb": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/recent_tweets_flood.ipynb",
        "file": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/recent_tweets_flood.csv",
    },
}
product = {
    "nb": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/tweets_timeline_hurricane.ipynb"
}


In [4]:
print('Disaster kind: ', disaster_title(disaster_kind))

Disaster kind:  Hurricane


## Load the sample of recent tweets

In [5]:
df_tweets = pd.read_csv(upstream[disaster_kind]['file'])
df_tweets.sample(5)

,author_id,tweet_text,created_at,hashtags,users
83027,89206778,"RT @sachikoko: 🎉🎈🎊RESCUED! 10YO BLACK &amp; WHITE KITTY ""FUZZ"" IS SAFE!🎉🎈🎊\n▶https://t.co/5I9ccVqF8H\n😺👏🏽PULLED BY Hurricane Pets Rescue Inc. (HP…",2022-07-31 14:30:52+00:00,[],['sachikoko']
26450,23521703,"With the busiest part of hurricane season fast approaching please join us tonight for ""Weathering the Storm"", a KATC Hurricane Special airing immediately after our 5pm newscast at 5:30pm! #LAwx https://t.co/atuLp6ECuE",2022-08-04 19:01:40+00:00,['LAwx'],[]
47185,966783341658824704,Has your boat been damaged by a major storm or hurricane? \nREAD! Tips For Recovering Your Boat After A Hurricane - https://t.co/qSchKwoJWJ #boatrecovery #hurricanedamage,2022-08-02 16:35:28+00:00,"['boatrecovery', 'hurricanedamage']",[]
28438,1303204457619038209,I see you standing in the rain. Within a storm—forever warm. Powerful as a hurricane. https://t.co/1ZO57nenuT,2022-08-04 18:16:20+00:00,[],[]
87546,1459763771149332481,Need a car https://t.co/NhVKPca4yF hi I'm William m Huber live in Florida it's hurricane season I'm raise money for car the price goal is not total of the car it's insurance tags cheap car mantaince everyone welcome read it and see if can donate this car will keep me safe,2022-08-01 10:31:53+00:00,[],[]


## Extract the locations from the tweet sample

In [6]:
%%time
df_tweets, df_locs_most_common = extract_assign_location(df_tweets, number_most_common=10)

df_tweets.sample(5)

0it [00:00, ?it/s]

23it [00:00, 228.09it/s]

55it [00:00, 274.76it/s]

86it [00:00, 287.24it/s]

119it [00:00, 299.44it/s]

...

105693it [05:07, 343.47it/s]

105729it [05:07, 346.60it/s]

105761it [05:07, 343.72it/s]

CPU times: user 5min 21s, sys: 3.84 s, total: 5min 25s
Wall time: 5min 24s


,author_id,tweet_text,created_at,hashtags,users,location
58026,1234551014029697024,"Remember, just 1 inch of water can cause roughly $25,000 of damage to your home. You can't control the weather during #HurricaneSeason but you can prepare for it with flood insurance. https://t.co/mdeBmpOSwx https://t.co/LtrpBs8VA5",2022-08-04 16:07:04+00:00,['HurricaneSeason'],[],[]
4872,25220817,RT @mattdanielwx: Just in: NOAA continues to forecast an active 2022 Atlantic Hurricane Season. Roughly 86% of the season occurs AFTER Augu…,2022-08-04 15:20:48+00:00,[],['mattdanielwx'],[]
95217,264224010,"Hurricane Hazel…the storm, not the mayor. https://t.co/n2qTdICA12",2022-07-31 16:21:31+00:00,[],[],[]
36840,899788415251697664,"RT @weather_history: August 3, 2020:\n\nHurricane Isaias made a late-night landfall near Ocean Isle Beach, NC. It was the second of 6 hurrica…",2022-08-03 18:07:58+00:00,[],['weather_history'],[Ocean Isle Beach]
52981,1240393957630185480,RT @NWSSanAntonio: La Niña is forecast to persist into the Fall. Did you know tropical cyclone activity in the Gulf increases during La Niñ…,2022-08-02 20:05:58+00:00,[],['NWSSanAntonio'],[Gulf increases]


In [7]:
df_locs_most_common

,location,count
0,Atlantic,7688
1,Gulf increases,2505
2,Florida,2212
3,California,1706
4,US,1387
5,Africa,1339
6,Gulf of Mexico,1247
7,Texas,1215
8,North Atlantic...Caribbean Sea,1164
9,Louisiana,1154


## Predict the class labels

In [8]:
# Get the vectorizer
with open(upstream["vectorizer"]["vectorizer"], 'rb') as f:
    vectorizer = pickle.load(f)

In [9]:
# Get the classification model
with open(upstream["category_classification_models"]["model_lr"], 'rb') as f:
    classifier = pickle.load(f)

In [10]:
%%time
# Predict the class labels
class_label = classifier.predict(vectorizer.transform(df_tweets['tweet_text']))
df_tweets['class_label'] = class_label

df_tweets.sample(5)

CPU times: user 28.2 s, sys: 117 ms, total: 28.4 s
Wall time: 28.4 s


,author_id,tweet_text,created_at,hashtags,users,location,class_label
35911,19426419,"RT @edgarrmcgregor: While it won't be hurricane Frank, this is a reminder that hurricanes can in fact hit southern California.\n\nThey are ve…",2022-08-01 04:15:10+00:00,[],['edgarrmcgregor'],[California],other_relevant_information
95768,4772894892,"RT @sachikoko: 🎉🎈🎊RESCUED! 10YO BLACK &amp; WHITE KITTY ""FUZZ"" IS SAFE!🎉🎈🎊\n▶https://t.co/5I9ccVqF8H\n😺👏🏽PULLED BY Hurricane Pets Rescue Inc. (HP…",2022-08-01 07:21:46+00:00,[],['sachikoko'],[],rescue_volunteering_or_donation_effort
88337,1548211665665019906,"@ACTBrigitte When there's a hurricane evacuation in Florida, how would tens of thousands of electric vehicles get out if they suddenly ran out of battery power and blocked the roads?",2022-08-02 07:40:55+00:00,[],['ACTBrigitte'],[Florida],displaced_people_and_evacuations
58884,1467858655617945611,RT @MarshGlobal: A1c: 2021’s Hurricane Ida caused storm surge of up to 14 feet along the Louisiana coast. Winds and storm surge lead to an…,2022-08-02 17:08:20+00:00,[],['MarshGlobal'],"[Louisiana, Winds]",other_relevant_information
66353,2210194536,【NOAA HURRICANE CENTER E N Pacific】Tropical Storm Frank Forecast Discussion Number 25 https://t.co/nfNdE4Emh7,2022-08-01 08:47:56+00:00,[],[],[],caution_and_advice


## Plot the intensity for each catagory, and for a particular location

### Prepare the time series grouped by disaster class label 

In [11]:
df_tweets['created_at'] = pd.to_datetime(df_tweets['created_at'], errors='coerce')
df_plot = df_tweets.copy()

df_plot['created_at'] = df_plot['created_at'].dt.floor('8H') # Round to 8 hours
df_plot['count'] = 1
df_group_plot = df_plot[['created_at', 'class_label', 'count']].groupby(['created_at', 'class_label']).count().reset_index()
df_group_plot['created_at'] = pd.to_datetime(df_group_plot['created_at'])
df_group_plot.head()

,created_at,class_label,count
0,2022-07-31 08:00:00+00:00,caution_and_advice,1168
1,2022-07-31 08:00:00+00:00,displaced_people_and_evacuations,361
2,2022-07-31 08:00:00+00:00,infrastructure_and_utility_damage,223
3,2022-07-31 08:00:00+00:00,not_humanitarian,407
4,2022-07-31 08:00:00+00:00,other_relevant_information,1251


### Prepare the time series grouped by location label 

In [12]:
df_explode = df_tweets.copy()

most_common10 = df_locs_most_common.head(10)['location'].tolist()

df_explode = df_explode.explode('location').reset_index(drop=True)
df_explode.dropna(inplace=True)
df_explode['created_at'] = df_explode['created_at'].dt.floor('8H') # Round to 8 hours
df_explode['count'] = 1
df_group_explode = df_explode[['created_at', 'location', 'count']].groupby(
    ['created_at', 'location']
).count().reset_index()
df_group_explode = df_group_explode[df_group_explode['location'].isin(most_common10)]
df_group_explode.head()

,created_at,location,count
0,2022-07-31 08:00:00+00:00,Africa,44
1,2022-07-31 08:00:00+00:00,Atlantic,152
3,2022-07-31 08:00:00+00:00,California,322
5,2022-07-31 08:00:00+00:00,Florida,44
6,2022-07-31 08:00:00+00:00,Gulf of Mexico,44


### Find the top 10 locations in the tweets sample

In [13]:
top10_locs = list(df_group_explode['location'].unique())
top10_locs

['Africa',
 'Atlantic',
 'California',
 'Florida',
 'Gulf of Mexico',
 'North Atlantic...Caribbean Sea',
 'Texas',
 'Louisiana',
 'US',
 'Gulf increases']

In [14]:
chart = alt.Chart(df_group_explode[df_group_explode['location'].isin(top10_locs)]).mark_line(size=2).encode(
    x='created_at:T',
    y='count:Q',
    color='location:N',
    tooltip=['location', 'count']
)

chart.properties(
    width=500,
    height=500
).properties(
    title=disaster_title(disaster_kind) + ' disasters in the world, interactive chart'
).interactive()

alt.Chart(...)

## Combine the most important locations per category

In [15]:
df_top_locations = top_locations(df_tweets, top10_locs)
df_top_locations.sample(5)

,created_at,class_label,locations,count
21,2022-08-01 02:00:00+00:00,other_relevant_information,"[Africa, California]",282
151,2022-08-06 10:00:00+00:00,caution_and_advice,[Africa],8
138,2022-08-05 14:00:00+00:00,not_humanitarian,[Atlantic],18
114,2022-08-04 14:00:00+00:00,infrastructure_and_utility_damage,"[Louisiana, Texas]",60
65,2022-08-02 22:00:00+00:00,caution_and_advice,"[Gulf of Mexico, North Atlantic...Caribbean Sea]",100


### Define the class labels we are interested in

In [16]:
class_labels = [
    'displaced_people_and_evacuations',
    'injured_or_dead_people',
    'rescue_volunteering_or_donation_effort',
    'requests_or_urgent_needs',
    'sympathy_and_support',
    'infrastructure_and_utility_damage',
    'caution_and_advice'
]

In [17]:
chart = alt.Chart(df_top_locations[df_top_locations['class_label'].isin(class_labels)]).mark_line(size=2).encode(
    x='created_at:T',
    y='count:Q',
    color='class_label:N',
    tooltip=['class_label:N', 'locations', 'count']
)

chart.properties(
    width=500,
    height=500
).properties(
    title=disaster_title(disaster_kind) + ' disasters in the world, interactive chart'
)

alt.Chart(...)

## Show sample tweets per most important locations above

In [18]:
df_top_location_tweets = top_locations_tweets(df_tweets, top10_locs)
df_top_location_tweets.sample(5)

,created_at,class_label,tweet_text,locations,count
63,2022-08-02 18:00:00+00:00,rescue_volunteering_or_donation_effort,[I was proud to help the people in the Florida panhandle recover quickly after a Category 5 storm. The Senate should pass this bill to reimburse the interest on local government loans. https://t.co/mapez48H7h],[Florida],52
167,2022-08-07 02:00:00+00:00,other_relevant_information,"[Last Days of Last Island: The Hurricane of 1856, Louisiana's First Great Storm [K7H5GBB]\n\nhttps://t.co/Whf0vrm1cI, Carrie and the Great Storm: A Galveston Hurricane Survival Story (Girls Survive) [NSJVKAL]\n\nhttps://t.co/fsnj7XpES8, Tropical wave coming off Africa could d...",[Africa],1
99,2022-08-04 02:00:00+00:00,infrastructure_and_utility_damage,"[@LAMUNRO2 I can remember my first Hurricane. A child in Toronto at the time. In the fifties. Hurricane Hazel. Did so much damage., @mkramer Hurricane Audrey in southwest Louisiana 1957. My parents had the eye pass over their house. No significant damage or venomous snakes un...",[Louisiana],37
0,2022-07-31 10:00:00+00:00,caution_and_advice,[NHC Atlantic Outlook\nhttps://t.co/DHXo73dOr8\n\nZCZC MIATWOAT ALL\nTTAA00 KNHC DDHHMM\n\nTropical Weather Outlook\nNWS National Hurricane Center Miami FL\n800 AM EDT Sun Jul 31 2022\n\nFor the North Atlantic...Caribbean Sea and the Gulf of Mexico:\n\nTropical cyclone format...,"[Gulf of Mexico, North Atlantic...Caribbean Sea]",88
34,2022-08-01 16:00:00+00:00,other_relevant_information,"[@ViewDownHere Never been in a hurricane but my first tropical storm in Jacksonville was gnarly. Coming from San Diego I was amazed at the amount of rain that dropped and glad all those buildings in Lejuene we're made of brick😂, Reporting from the Eye of the Hurricane: A 75 C...",[California],120


In [19]:
show_top_locations(df_top_location_tweets, class_labels, disaster_kind)

alt.VConcatChart(...)

## Visualize the locations and relationship between them as a network

### Select the top 20 locations for this disaster

In [20]:
loc20 = get_top_locations_flatten(df_tweets, 20)
loc20

['Atlantic',
 'Gulf increases',
 'Florida',
 'California',
 'US',
 'Africa',
 'Gulf of Mexico',
 'Texas',
 'North Atlantic...Caribbean Sea',
 'Louisiana',
 'Gambia',
 'Independence Stadi…',
 'New Orleans',
 'St',
 'Petersburg',
 'Gulf Coast',
 'Ocean Isle Beach',
 'National Hurricane Center',
 'Kentucky',
 'North Pacific']

### Get the tweets for just the top 20 locations

In [21]:
df_loc_graph = get_location_graph_data(df_tweets, loc20)
df_loc_graph.sample(10)

,class_label,location_x,location_y,count
9,other_relevant_information,National Hurricane Center,Atlantic,73
22,rescue_volunteering_or_donation_effort,Atlantic,Africa,42
15,other_relevant_information,Texas,Gulf Coast,109
14,other_relevant_information,St,Petersburg,818
11,other_relevant_information,New Orleans,Louisiana,52
24,rescue_volunteering_or_donation_effort,Louisiana,Kentucky,54
5,other_relevant_information,Gambia,Africa,1090
10,other_relevant_information,New Orleans,Gulf Coast,212
6,other_relevant_information,Gulf Coast,Florida,11
0,caution_and_advice,North Atlantic...Caribbean Sea,Gulf of Mexico,1164


In [22]:
plot_disaster_mentions(df_loc_graph, 'caution_and_advice', 'Caution And Advice')

alt.LayerChart(...)

### Get the most frequent location pair

In [23]:
loc_pairs = top_location_pairs(df_loc_graph, 'caution_and_advice', top_pairs_count=1)
loc_pairs

[['North Atlantic...Caribbean Sea', 'Gulf of Mexico']]

### Show a sample for this location pair

In [24]:
show_tweets_locations(df_tweets, class_label='caution_and_advice' , locations=loc_pairs[0], sample=5)

,tweet_text,location
0,NHC Atlantic Outlook\nhttps://t.co/DHXo73dOr8\n\nZCZC MIATWOAT ALL\nTTAA00 KNHC DDHHMM\n\nTropical Weather Outlook\nNWS National Hurricane Center Miami FL\n800 AM EDT Mon Aug 1 2022\n\nFor the North Atlantic...Caribbean Sea and the Gulf of Mexico:\n\nTropical cyclone formatio...,"[North Atlantic...Caribbean Sea, Gulf of Mexico]"
1,NHC Atlantic Outlook\nhttps://t.co/DHXo72Wd2y\n\nZCZC MIATWOAT ALL\nTTAA00 KNHC DDHHMM\n\nTropical Weather Outlook\nNWS National Hurricane Center Miami FL\n200 AM EDT Tue Aug 2 2022\n\nFor the North Atlantic...Caribbean Sea and the Gulf of Mexico:\n\nTropical cyclone formatio...,"[North Atlantic...Caribbean Sea, Gulf of Mexico]"
2,NHC Atlantic Outlook\nhttps://t.co/DHXo72Wd2y\n\nZCZC MIATWOAT ALL\nTTAA00 KNHC DDHHMM\n\nTropical Weather Outlook\nNWS National Hurricane Center Miami FL\n200 AM EDT Tue Aug 2 2022\n\nFor the North Atlantic...Caribbean Sea and the Gulf of Mexico:\n\nTropical cyclone formatio...,"[North Atlantic...Caribbean Sea, Gulf of Mexico]"
3,NHC Atlantic Outlook\nhttps://t.co/DHXo73dOr8\n\nZCZC MIATWOAT ALL\nTTAA00 KNHC DDHHMM\n\nTropical Weather Outlook\nNWS National Hurricane Center Miami FL\n200 PM EDT Wed Aug 3 2022\n\nFor the North Atlantic...Caribbean Sea and the Gulf of Mexico:\n\nTropical cyclone formatio...,"[North Atlantic...Caribbean Sea, Gulf of Mexico]"
4,Tropical Weather Outlook\nNWS National Hurricane Center Miami FL\nWed Aug 3 2022\n\nFor the North Atlantic...Caribbean Sea and the Gulf of Mexico:\n\nTropical cyclone formation is not expected during the next 5 days. https://t.co/ob8FUJ9ZDE,"[North Atlantic...Caribbean Sea, Gulf of Mexico]"


In [25]:
plot_disaster_mentions(df_loc_graph, 'displaced_people_and_evacuations', 'Displaced People And Evacuations')

alt.LayerChart(...)

### Get the most frequent location pair

In [26]:
loc_pairs = top_location_pairs(df_loc_graph, 'displaced_people_and_evacuations', top_pairs_count=1)
loc_pairs

[['New Orleans', 'Louisiana']]

### Show a sample for this location pair

In [27]:
show_tweets_locations(df_tweets, class_label='displaced_people_and_evacuations' , locations=loc_pairs[0], sample=5)

,tweet_text,location
0,"We're also partnering with @Kalshi in New Orleans.\n\nIn recent years, as the climate emergency has accelerated, Louisiana has borne the brunt of hurricane season year after year. Kalshi can help in the event of a last-minute evacuation on a tight budget.\nhttps://t.co/oMUWLr...","[New Orleans, Louisiana]"
1,"We're also partnering with @Kalshi in New Orleans.\n\nIn recent years, as the climate emergency has accelerated, Louisiana has borne the brunt of hurricane season year after year. Kalshi can help in the event of a last-minute evacuation on a tight budget.\nhttps://t.co/oMUWLr...","[New Orleans, Louisiana]"
2,"We're also partnering with @Kalshi in New Orleans.\n\nIn recent years, as the climate emergency has accelerated, Louisiana has borne the brunt of hurricane season year after year. Kalshi can help in the event of a last-minute evacuation on a tight budget.\nhttps://t.co/oMUWLr...","[New Orleans, Louisiana]"
3,"We're also partnering with @Kalshi in New Orleans.\n\nIn recent years, as the climate emergency has accelerated, Louisiana has borne the brunt of hurricane season year after year. Kalshi can help in the event of a last-minute evacuation on a tight budget.\nhttps://t.co/oMUWLr...","[New Orleans, Louisiana]"
4,"We're also partnering with @Kalshi in New Orleans.\n\nIn recent years, as the climate emergency has accelerated, Louisiana has borne the brunt of hurricane season year after year. Kalshi can help in the event of a last-minute evacuation on a tight budget.\nhttps://t.co/oMUWLr...","[New Orleans, Louisiana]"


In [28]:
plot_disaster_mentions(df_loc_graph, 'injured_or_dead_people', 'Injured Or Dead People')

'No records found for class injured_or_dead_people'

### Get the most frequent location pair

In [29]:
loc_pairs = top_location_pairs(df_loc_graph, 'injured_or_dead_people', top_pairs_count=1)
loc_pairs

[[]]

### Show a sample for this location pair

In [30]:
show_tweets_locations(df_tweets, class_label='injured_or_dead_people' , locations=loc_pairs[0], sample=5)

'No location pairs provided'

In [31]:
plot_disaster_mentions(df_loc_graph, 'rescue_volunteering_or_donation_effort',
                       'Rescue Volunteering Or Donation Effort')

alt.LayerChart(...)

### Get the most frequent location pair

In [32]:
loc_pairs = top_location_pairs(df_loc_graph, 'rescue_volunteering_or_donation_effort', top_pairs_count=1)
loc_pairs

[['Kentucky', 'Florida']]

### Show a sample for this location pair

In [33]:
show_tweets_locations(df_tweets, class_label='rescue_volunteering_or_donation_effort',
                      locations=loc_pairs[0], sample=5)

,tweet_text,location
0,RT @MAndersen66: @Acyn Radical social agenda:\n\nHelping flood victims in Kentucky.\n\nHelping hurricane victims in Florida.\n\nHelping storm vic…,"[Kentucky, Florida]"
1,"@Acyn Radical social agenda:\n\nHelping flood victims in Kentucky.\n\nHelping hurricane victims in Florida.\n\nHelping storm victims in Nebraska.\n\nHelping landslide victims in Alaska.\n\nThat radical social agenda, Marsha?\n\nWhy don't YOU want to help people?","[Kentucky, Florida, Nebraska., Alaska]"
2,RT @MAndersen66: @Acyn Radical social agenda:\n\nHelping flood victims in Kentucky.\n\nHelping hurricane victims in Florida.\n\nHelping storm vic…,"[Kentucky, Florida]"
3,RT @MAndersen66: @Acyn Radical social agenda:\n\nHelping flood victims in Kentucky.\n\nHelping hurricane victims in Florida.\n\nHelping storm vic…,"[Kentucky, Florida]"
4,RT @MAndersen66: @Acyn Radical social agenda:\n\nHelping flood victims in Kentucky.\n\nHelping hurricane victims in Florida.\n\nHelping storm vic…,"[Kentucky, Florida]"


In [34]:
plot_disaster_mentions(df_loc_graph, 'requests_or_urgent_needs',
                       'requests_or_urgent_needs')

'No records found for class requests_or_urgent_needs'

### Get the most frequent location pair

In [35]:
loc_pairs = top_location_pairs(df_loc_graph, 'requests_or_urgent_needs', top_pairs_count=1)
loc_pairs

[[]]

### Show a sample for this location pair

In [36]:
show_tweets_locations(df_tweets, class_label='requests_or_urgent_needs',
                      locations=loc_pairs[0], sample=5)

'No location pairs provided'

In [37]:
plot_disaster_mentions(df_loc_graph, 'sympathy_and_support',
                       'sympathy_and_support')

'No records found for class sympathy_and_support'

### Get the most frequent location pair

In [38]:
loc_pairs = top_location_pairs(df_loc_graph, 'sympathy_and_support', top_pairs_count=1)
loc_pairs

[[]]

### Show a sample for this location pair

In [39]:
show_tweets_locations(df_tweets, class_label='sympathy_and_support',
                      locations=loc_pairs[0], sample=5)

'No location pairs provided'

In [40]:
plot_disaster_mentions(df_loc_graph, 'infrastructure_and_utility_damage',
                       'infrastructure_and_utility_damage')

alt.LayerChart(...)

### Get the most frequent location pair

In [41]:
loc_pairs = top_location_pairs(df_loc_graph, 'infrastructure_and_utility_damage', top_pairs_count=1)
loc_pairs

[['US', 'Texas']]

### Show a sample for this location pair

In [42]:
show_tweets_locations(df_tweets, class_label='infrastructure_and_utility_damage',
                      locations=loc_pairs[0], sample=5)

,tweet_text,location
0,"@SenJoniErnst @SenTedCruz demanded aid for Texas after the Texas Power Grid Collapse, but attacked giving aid to New England after Superstorm Sandy,\nwhich did $65 BILLION in damage/killed 285 AMERICANS or Puerto Rico after Hurricane Maria did $90 BILLION in damage/killed 4,6...","[Texas, New England, Puerto Rico, US]"
1,"@SenJoniErnst @SenTedCruz demanded aid for Texas after the Texas Power Grid Collapse, but attacked giving aid to New England after Superstorm Sandy,\nwhich did $65 BILLION in damage/killed 285 AMERICANS or Puerto Rico after Hurricane Maria did $90 BILLION in damage/killed 4,6...","[Texas, New England, Puerto Rico, US]"
2,"@SenJoniErnst @SenTedCruz demanded aid for Texas after the Texas Power Grid Collapse, but attacked giving aid to New England after Superstorm Sandy,\nwhich did $65 BILLION in damage/killed 285 AMERICANS or Puerto Rico after Hurricane Maria did $90 BILLION in damage/killed 4,6...","[Texas, New England, Puerto Rico, US]"
3,"@tedcruz @SenTedCruz demanded aid for Texas after the Texas Power Grid Collapse, but attacked giving aid to New England after Superstorm Sandy,\nwhich did $65 BILLION in damage/killed 285 AMERICANS or Puerto Rico after Hurricane Maria did $90 BILLION in damage/killed 4,645 of...","[Texas, New England, Puerto Rico, US]"
4,"@tedcruz @SenTedCruz demanded aid for Texas after the Texas Power Grid Collapse, but attacked giving aid to New England after Superstorm Sandy,\nwhich did $65 BILLION in damage/killed 285 AMERICANS or Puerto Rico after Hurricane Maria did $90 BILLION in damage/killed 4,645 of...","[Texas, New England, Puerto Rico, US]"
